In [ ]:
!pip install gymnasium[atari]
!pip install gymnasium matplotlib numpy
!pip install ale-py
!pip install gymnasium[accept-rom-license]
!pip install tensorflow


: 

In [2]:
from ale_py import ALEInterface
from ale_py.roms import Breakout
import gymnasium as gym
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.optimizers import Adam

KeyboardInterrupt: 

In [ ]:
ale = ALEInterface()
ale.loadROM(Breakout)

In [ ]:
import random
env = gym.make("BreakoutNoFrameskip-v4", render_mode="human")
observation, info = env.reset()

next_frame, reward, terminated, truncated, info = env.step(1)
print('Reward Recieved = ' + str(reward))
print('Next state is a terminal state: ' + str(terminated))
print('info[ale.lives] tells us how many lives we have. Lives: ' + str(info['lives']))
print(env.reward_range)


for i in range(10000):
    a = random.sample([0,1,2,3] , 1)[0]
    next_frame, reward, terminated, truncated, info = env.step(a)
    print(reward)
    env.render()
    if truncated == True:
        env.reset()
        

Reward Recieved = 0.0
Next state is a terminal state: False
info[ale.lives] tells us how many lives we have. Lives: 5
(-inf, inf)
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


C:\Users\xuhel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\gymnasium\utils\passive_env_checker.py:365: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  "No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps."


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


: 

In [ ]:
#keeps track of the agents memory
def Agent_memory():
    def __init__(self):
        self.action_memory = []
        self.frame_memory = []
        self.reward_memory = []
        self.next_state_memory = []
#function to keep adding to the memory
    def add_memory(self, next_frame, next_frames_reward, next_action):
        self.frame_memory.append(next_frame)
        self.action_memory.append(next_action)
        self.reward_memory.append(next_frames_reward)
        #self.done_flags.append(next_frame_terminal)
        
# s to s-, need ot know all possible values of s dash and the probablity matrix,
# for each state and all possibnle acto what is the prbablity of going to next staticmethod
#     function promixater, you tell it what state, what action u are taking and tell u what the value of ur next state is
# that has to be train, the function promixer is the NN, action value function. try to find a value action function. 
#given the state, how can we find these variables, the more state space more complex, x, y, vx
#define state space, define actions, we have to define gamma, 

In [ ]:
learning_rate = 0.5
epsilon = 0.99
gamma = 0.99
episodes = 100
max_timesteps = 10

In [1]:
class Agent():
    def __init__(self, possible_actions, starting_mem_len, max_mem_len, starting_epsilon, learn_rate, starting_lives = 5, debug = False) -> None:
       pass
    '''gets the next set of actions from what the agent learns'''
    def get_action(self):
        pass
    def learn(self):
        pass
    